In [ ]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd
from tensorflow import keras

### tf.data.Dataset.from_tensor_slices()：在RAM中完全创建一个数据集。采用一个tensor并创建一个tf.data.Dataset(其元素都是X沿着第一维的切片)

In [ ]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
#这个数据集包含10个元素，张量0、1、2、3、...、10
dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [ ]:
for item in dataset:
  print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [ ]:
#相同效果
dataset_ = tf.data.Dataset.range(10)
#这个数据集包含10个元素，张量0、1、2、3、...、9
dataset_

<RangeDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [ ]:
for item in dataset_:
  print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


# 链式转换


通过数据集的转换方法对其进行各种转换，每次运用转换方法都返回一个新的数据集

In [ ]:
dataset_1 = dataset.repeat(3).batch(7)
#原始数据集调用repeat（3）后返回一个新数据集，它将重复原始数据集三次
#在新数据集上调用batch（）后了再次创建一个新数据集，它将先前数据集的元素以7个元素为一个批次分组
for item in dataset_1:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


设置drop_remainder=True使得所有批次具有相同的大小

In [ ]:
dataset_2 = dataset.repeat(3).batch(7,drop_remainder=True)
#把最后一个批次（只含有两个元素）删除
for item in dataset_2:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)


#### map()：用于变换元素，应用于每个元素。
传递给map()的函数必须可转换为TF函数。可调用map()把预处理应用于数据。

In [ ]:
dataset_3 = dataset.map(lambda x: x * 2)#创建一个新训练集，所有元素是原始元素的两倍
for item in dataset_3:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(18, shape=(), dtype=int32)


### filter():简单过滤数据集

In [ ]:
dataset_4 = dataset_3.filter(lambda x: x < 10)  # keep only items < 10
for item in dataset_4:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)


### take()：查看数据集中的一些元素

In [ ]:
for item in dataset_4.take(3):#查看前三个元素
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


### shuffle(buffer_size,seed=None,reshuffle_each_iteration=None)：将序列的所有元素随机排序。将第一个元素填到缓冲区，要求提供元素时从缓存区随机取出一个元素并用源数据集中新元素替换。
参数buffer_size:缓存区大小。

buffer_size=1:不打乱顺序，既保持原序。buffer_size越大，打乱程度越大

应该先shuffle再batch，如果先batch后shuffle的话，那么此时就只是对batch进行shuffle，而batch里面的数据顺序依旧是有序的，那么随机程度会减弱(实际并未shuffle)

In [ ]:
tf.random.set_seed(42)
#随机混洗数据集的元素
dataset = tf.data.Dataset.range(10).repeat(3)
#每3个元素进行一次shuffle
dataset = dataset.shuffle(buffer_size=3, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([1 3 0 4 2 5 6], shape=(7,), dtype=int64)
tf.Tensor([8 7 1 0 3 2 5], shape=(7,), dtype=int64)
tf.Tensor([4 6 9 8 9 7 0], shape=(7,), dtype=int64)
tf.Tensor([3 1 4 5 2 8 7], shape=(7,), dtype=int64)
tf.Tensor([6 9], shape=(2,), dtype=int64)


# 乱序数据

如果想要每次都有相同的随机排序，可提供相同的随机种子

In [ ]:
tf.random.set_seed(42)
dataset_5 = tf.data.Dataset.range(10).repeat(3)
dataset_5 = dataset_5.shuffle(buffer_size=5, seed=42).batch(7)
#相当于没动
for item in dataset_5:
    print(item)

tf.Tensor([0 1 6 5 7 3 9], shape=(7,), dtype=int64)
tf.Tensor([8 2 1 0 4 6 4], shape=(7,), dtype=int64)
tf.Tensor([7 2 5 9 2 1 3], shape=(7,), dtype=int64)
tf.Tensor([4 3 8 7 9 5 0], shape=(7,), dtype=int64)
tf.Tensor([8 6], shape=(2,), dtype=int64)


In [ ]:
tf.random.set_seed(42)
dataset_6 = tf.data.Dataset.range(10).repeat(3)
dataset_6 = dataset_6.shuffle(buffer_size=5, seed=42,reshuffle_each_iteration=False).batch(7)
for item in dataset_6:
    print(item)

tf.Tensor([3 4 2 7 0 6 9], shape=(7,), dtype=int64)
tf.Tensor([0 1 5 1 5 8 2], shape=(7,), dtype=int64)
tf.Tensor([3 6 8 0 9 2 1], shape=(7,), dtype=int64)
tf.Tensor([7 6 4 5 4 8 9], shape=(7,), dtype=int64)
tf.Tensor([3 7], shape=(2,), dtype=int64)


#### 在乱序中的数据集上调用repeat()默认将在每次迭代时生成一个新次序，shuffle()设置reshuffle_each_iteration=False可以使其在每次迭代中重用相同的顺序

In [ ]:
#重复时生成新次序
dataset_5_2 = dataset_5.repeat(2)
for item in dataset_5_2:
    print(item)

tf.Tensor([3 4 0 1 2 8 9], shape=(7,), dtype=int64)
tf.Tensor([1 0 2 6 4 7 3], shape=(7,), dtype=int64)
tf.Tensor([5 7 8 9 2 5 1], shape=(7,), dtype=int64)
tf.Tensor([6 4 5 3 8 6 9], shape=(7,), dtype=int64)
tf.Tensor([7 0], shape=(2,), dtype=int64)
tf.Tensor([1 3 4 7 8 2 5], shape=(7,), dtype=int64)
tf.Tensor([0 0 2 1 6 9 5], shape=(7,), dtype=int64)
tf.Tensor([4 7 0 1 8 9 6], shape=(7,), dtype=int64)
tf.Tensor([5 2 7 6 4 9 8], shape=(7,), dtype=int64)
tf.Tensor([3 3], shape=(2,), dtype=int64)


In [ ]:
#重复相同顺序
dataset_6_2 = dataset_6.repeat(2)
for item in dataset_6_2:
    print(item)

tf.Tensor([3 4 2 7 0 6 9], shape=(7,), dtype=int64)
tf.Tensor([0 1 5 1 5 8 2], shape=(7,), dtype=int64)
tf.Tensor([3 6 8 0 9 2 1], shape=(7,), dtype=int64)
tf.Tensor([7 6 4 5 4 8 9], shape=(7,), dtype=int64)
tf.Tensor([3 7], shape=(2,), dtype=int64)
tf.Tensor([3 4 2 7 0 6 9], shape=(7,), dtype=int64)
tf.Tensor([0 1 5 1 5 8 2], shape=(7,), dtype=int64)
tf.Tensor([3 6 8 0 9 2 1], shape=(7,), dtype=int64)
tf.Tensor([7 6 4 5 4 8 9], shape=(7,), dtype=int64)
tf.Tensor([3 7], shape=(2,), dtype=int64)


# 大型数据集混洗
将源数据拆分为多个文件，在训练过程中以随机顺序读取它们；为避免同一文件中的实例相互接近，可随机选择多个文件并同时读取、交错它们的记录；还可使用shuffle()方法添加一个乱序缓存区。

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
#8个输入特征，一个标签即房屋价值中间值
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()#归一化
scaler.fit(X_train)
X_mean = scaler.mean_#均值
X_std = scaler.scale_#方差

In [ ]:
print("X_train:",X_train.shape)
print("y_train:",y_train.shape)
print("X_test:",X_test.shape)
print("y_test:",y_test.shape)
print("X_valid:",X_valid.shape)
print("y_valid:",y_valid.shape)

X_train: (11610, 8)
y_train: (11610, 1)
X_test: (5160, 8)
y_test: (5160, 1)
X_valid: (3870, 8)
y_valid: (3870, 1)


In [ ]:
X_train[0,:]

array([ 3.52140000e+00,  1.50000000e+01,  3.04994451e+00,  1.10654828e+00,
        1.44700000e+03,  1.60599334e+00,  3.76300000e+01, -1.22430000e+02])

#### repr():返回一个对象的 string 格式。

## 将训练集、测试集、验证集都分成许多csv文件

#### **os.path.join()**：用于路径拼接文件路径，可以传入多个路径
#### **os.makedirs()**: 递归创建目录
#### **enumerate()**:将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标
#### **with open()**的wt模式：读写模式，写文件时会用\r\n来表示换行。
#### **repr()**:返回一个对象的 string 格式。

In [ ]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    """将数据集分成多个csv文件"""
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)#创建路径
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")#生成文件名

    filepaths = []#包含文件路径的列表
    m = len(data)#数据集所有数据总数
    #np.array_split（）将创建的包含数字1-m的数据集切分成n_parts个数组，并给切分出的每个数组索引
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            #把文件中每一行的数据转化为string格式，在两行中间加入逗号和分行符
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths#文件名

#### np.c_():是np.r_['-1,2,0', index expression] 的一种简写，沿着矩阵的第二个轴拼接

In [ ]:
#将x、y拼接做成原始数据集
train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]#添加一行文件头（8个标签的描述和target描述）
header = ",".join(header_cols)#将文件头用逗号和下一行分开

In [ ]:
print("train_data:",train_data.shape)
print("valid_data:",valid_data.shape)
print("test_data:",test_data.shape)

train_data: (11610, 9)
valid_data: (3870, 9)
test_data: (5160, 9)


In [ ]:
train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
#将训练集分成20个csv，每个文件都包含定义好的文件头
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)

In [ ]:
train_filepaths[0]#第一个切分的文件名为my_train_00.csv

'datasets/housing/my_train_00.csv'

### 使用pd.read_csv（）查看训练集分出的第一个csv文件

In [ ]:
pd.read_csv(train_filepaths[0]).head()#（显示前5个实例）

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,3.5214,15.0,3.049945,1.106548,1447.0,1.605993,37.63,-122.43,1.442
1,5.3275,5.0,6.490060,0.991054,3464.0,3.443340,33.69,-117.39,1.687
2,3.1000,29.0,7.542373,1.591525,1328.0,2.250847,38.44,-122.98,1.621
3,7.1736,12.0,6.289003,0.997442,1054.0,2.695652,33.55,-117.70,2.621
4,2.0549,13.0,5.312457,1.085092,3297.0,2.244384,33.93,-116.93,0.956


In [ ]:
type(train_filepaths)

list

In [ ]:
pd.read_csv(train_filepaths[1]).head()#第二个训练集文件的数据

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,3.6875,44.0,4.524476,0.993007,457.0,3.195804,34.04,-118.15,1.625
1,3.9917,25.0,5.576364,1.034545,1602.0,2.912727,38.33,-122.82,2.441
2,1.6900,18.0,4.423529,1.145882,994.0,2.338824,34.14,-116.32,0.542
3,3.9643,52.0,4.797980,1.020202,467.0,2.358586,37.84,-122.26,1.888
4,6.6704,25.0,6.829317,1.022088,1503.0,3.018072,33.88,-117.99,2.406


In [ ]:
pd.read_csv(train_filepaths[1])

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,3.6875,44.0,4.524476,0.993007,457.0,3.195804,34.04,-118.15,1.625
1,3.9917,25.0,5.576364,1.034545,1602.0,2.912727,38.33,-122.82,2.441
2,1.6900,18.0,4.423529,1.145882,994.0,2.338824,34.14,-116.32,0.542
3,3.9643,52.0,4.797980,1.020202,467.0,2.358586,37.84,-122.26,1.888
4,6.6704,25.0,6.829317,1.022088,1503.0,3.018072,33.88,-117.99,2.406
...,...,...,...,...,...,...,...,...,...
576,4.3148,25.0,5.310108,0.979392,2243.0,2.201178,34.12,-117.75,2.407
577,1.4063,51.0,4.446429,1.026786,368.0,3.285714,34.06,-117.73,0.988
578,3.6815,17.0,4.765864,1.043764,1077.0,2.356674,33.82,-118.03,2.453
579,2.1471,33.0,4.313131,1.186869,645.0,3.257576,33.99,-118.47,2.393


In [ ]:
len(train_filepaths)#共将训练集数据（含11610个实例）分成了20个csv文件（581个实例）

20

### 使用文件模式查看训练集分出的第一个csv文件

In [ ]:
with open(train_filepaths[0]) as f:
    for i in range(5):
        print(f.readline(), end="")

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442
5.3275,5.0,6.490059642147117,0.9910536779324056,3464.0,3.4433399602385686,33.69,-117.39,1.687
3.1,29.0,7.5423728813559325,1.5915254237288134,1328.0,2.2508474576271187,38.44,-122.98,1.621
7.1736,12.0,6.289002557544757,0.9974424552429667,1054.0,2.6956521739130435,33.55,-117.7,2.621


## 创建一个仅包含以下文件路径的数据集,打乱文件夹中文件路径的顺序

In [ ]:
train_filepaths

['datasets/housing/my_train_00.csv',
 'datasets/housing/my_train_01.csv',
 'datasets/housing/my_train_02.csv',
 'datasets/housing/my_train_03.csv',
 'datasets/housing/my_train_04.csv',
 'datasets/housing/my_train_05.csv',
 'datasets/housing/my_train_06.csv',
 'datasets/housing/my_train_07.csv',
 'datasets/housing/my_train_08.csv',
 'datasets/housing/my_train_09.csv',
 'datasets/housing/my_train_10.csv',
 'datasets/housing/my_train_11.csv',
 'datasets/housing/my_train_12.csv',
 'datasets/housing/my_train_13.csv',
 'datasets/housing/my_train_14.csv',
 'datasets/housing/my_train_15.csv',
 'datasets/housing/my_train_16.csv',
 'datasets/housing/my_train_17.csv',
 'datasets/housing/my_train_18.csv',
 'datasets/housing/my_train_19.csv']

#### **tf.data.Dataset.list_files (file_pattern)**：用于获取文件路径列表，可获得文件夹下面的全部指定文件，file_pattern使用正则路径。返回一个乱序的文件路径的数据集。（若不希望乱序可设置shuffle=False）

In [ ]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

In [ ]:
for item in filepath_dataset:
  print(item)

tf.Tensor(b'datasets/housing/my_train_15.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_08.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_03.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_19.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_02.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_09.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_00.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_07.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_12.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_04.csv', shape=(), dtype=string)
tf.Ten

## 一次读取5个文件、交织它们的行（使用skip（）跳过每个文件的第一行即标题行）

### 最好使用具有相同长度的文件，否则最长文件的结尾将不会交织

#### **tf.data.Dataset.interleave(map_func, cycle_length=AUTOTUNE, block_length=1)**：返回一个新数据集。
假定我们现在有一个数据集A。从该A中取出cycle_length个元素，然后对这些元素应用map_func方法得到cycle_length个新的Dataset对象。然后从这些新生成的Dataset对象中取数据，逻辑为轮流从每个对象里面取数据，每次取block_length个。当这些新生成的某个Dataset的对象取尽时，从原Dataset中再取cycle_length个元素，然后应用map_func，以此类推。

#### **tf.data.TextLineDataset(filenames，buffer_size)**:一种从数据文件中读取的方法。只需提供文件名（1个或者多个），这个接口会自动构造一个dataset。类中保存的元素：文中一行，就是一个元素，是string类型的tensor。
filenames:单个或者多个string格式的文件名或者目录；
buffer_size:决定缓冲字节数多少

filepath_dataset:文件路径数据集，1个

dataset:交织数据集，1个

TextLineDataset：交织数据集在内部创建的数据集，共5个


In [ ]:
n_readers = 5
#interleave()从filepath_dataset取5个文件路径，对每个路径应用lambda方法来创建新数据集
#TextLineDataset（）从filepath文件夹中读取数据文件
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)

#### interleave示例

In [ ]:
#interleave示例
a = tf.data.Dataset.range(1, 6)  # ==> [ 1, 2, 3, 4, 5 ]

#取2个元素：1、2;重复6遍→[[1,1,1,1,1,1],[2,2,2,2,2,2]],在其中轮流从每个对象里每次取4个数据
#→第一次[[1,1,1,1],第二次[2,2,2,2]]，第三次[1,1,2,2]
#此时缓存区元素取尽，从数据集中取3、4;重复6遍→...
b=a.interleave(lambda x: tf.data.Dataset.from_tensors(x).repeat(6),
            cycle_length=2, block_length=4) 
for item in b:
    print(item.numpy(),end=', ')

1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 3, 3, 4, 4, 5, 5, 5, 5, 5, 5, 

### 遍历交织数据集dataset

循环遍历5个TextLineDatasets，每次读取一行直至所有数据集都读出；然后它从filepath获取剩下的15个文件中前5个文件的路径，以相同方法对它们进行交织，直至读完所有文件的路径。

In [ ]:
#交织数据集中前5个元素（即随机选择的5个csv文件的第一行（忽略标题行））
#第一个元素是第一个读取的文件的第一个元素，第二个元素是第二个读取的文件的第一个元素...
for line in dataset.take(5):
    print(line.numpy())

b'4.6477,38.0,5.03728813559322,0.911864406779661,745.0,2.5254237288135593,32.64,-117.07,1.504'
b'8.72,44.0,6.163179916317992,1.0460251046025104,668.0,2.794979079497908,34.2,-118.18,4.159'
b'3.8456,35.0,5.461346633416459,0.9576059850374065,1154.0,2.8778054862842892,37.96,-122.05,1.598'
b'3.3456,37.0,4.514084507042254,0.9084507042253521,458.0,3.2253521126760565,36.67,-121.7,2.526'
b'3.6875,44.0,4.524475524475524,0.993006993006993,457.0,3.195804195804196,34.04,-118.15,1.625'
